# Persistence

In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gemma3:4b",
    model_provider="ollama",
    temperature = 0
    )

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza break up with the bread? \n\n... Because it said, "You\'re too cheesy!" 😂 \n\n---\n\nWould you like to hear another pizza joke?',
 'explanation': 'That’s a classic! The joke plays on the double meaning of “cheesy.” \n\n* **Literally:** Bread can be made with cheese, so the pizza is saying the bread is *too* full of cheese.\n* **Figuratively:** “Cheesy” is also a slang term for someone overly sentimental or clingy. The pizza is saying the bread is too clingy!\n\nIt’s a cute and silly joke! 😄\n\n---\n\nYes, please! I’d love to hear another pizza joke.'}

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza break up with the bread? \n\n... Because it said, "You\'re too cheesy!" 😂 \n\n---\n\nWould you like to hear another pizza joke?', 'explanation': 'That’s a classic! The joke plays on the double meaning of “cheesy.” \n\n* **Literally:** Bread can be made with cheese, so the pizza is saying the bread is *too* full of cheese.\n* **Figuratively:** “Cheesy” is also a slang term for someone overly sentimental or clingy. The pizza is saying the bread is too clingy!\n\nIt’s a cute and silly joke! 😄\n\n---\n\nYes, please! I’d love to hear another pizza joke.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-e6b2-6d43-8002-d623611d2675'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-22T16:29:11.370461+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-b427-60c9-8001-bfd3ccf1edc7'}}, tasks=(), i

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza break up with the bread? \n\n... Because it said, "You\'re too cheesy!" 😂 \n\n---\n\nWould you like to hear another pizza joke?', 'explanation': 'That’s a classic! The joke plays on the double meaning of “cheesy.” \n\n* **Literally:** Bread can be made with cheese, so the pizza is saying the bread is *too* full of cheese.\n* **Figuratively:** “Cheesy” is also a slang term for someone overly sentimental or clingy. The pizza is saying the bread is too clingy!\n\nIt’s a cute and silly joke! 😄\n\n---\n\nYes, please! I’d love to hear another pizza joke.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-e6b2-6d43-8002-d623611d2675'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-22T16:29:11.370461+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-b427-60c9-8001-bfd3ccf1edc7'}}, tasks=(), 

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti break up with the meatball? \n\n... Because he was too clingy! 🍝❤️ \n\n---\n\nWould you like to hear another pasta joke?',
 'explanation': 'That’s a cute and classic joke! The humor comes from the double meaning of “clingy.” \n\n* **Literally:** Meatballs are known for sticking to pasta dishes.\n* **Figuratively:** “Clingy” describes someone who is overly attached or demanding in a relationship.\n\nIt’s a silly play on words!\n\n---\n\nYes, please! I’d love to hear another pasta joke. 😄'}

In [15]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the meatball? \n\n... Because he was too clingy! 🍝❤️ \n\n---\n\nWould you like to hear another pasta joke?', 'explanation': 'That’s a cute and classic joke! The humor comes from the double meaning of “clingy.” \n\n* **Literally:** Meatballs are known for sticking to pasta dishes.\n* **Figuratively:** “Clingy” describes someone who is overly attached or demanding in a relationship.\n\nIt’s a silly play on words!\n\n---\n\nYes, please! I’d love to hear another pasta joke. 😄'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100ba-23ee-67b7-8002-af419e444b52'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-22T16:29:17.791192+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-f9e6-6d70-8001-6ef8dcd99dd9'}}, tasks=(), interrupts=())

In [16]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the meatball? \n\n... Because he was too clingy! 🍝❤️ \n\n---\n\nWould you like to hear another pasta joke?', 'explanation': 'That’s a cute and classic joke! The humor comes from the double meaning of “clingy.” \n\n* **Literally:** Meatballs are known for sticking to pasta dishes.\n* **Figuratively:** “Clingy” describes someone who is overly attached or demanding in a relationship.\n\nIt’s a silly play on words!\n\n---\n\nYes, please! I’d love to hear another pasta joke. 😄'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100ba-23ee-67b7-8002-af419e444b52'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-22T16:29:17.791192+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-f9e6-6d70-8001-6ef8dcd99dd9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'jo

# Time Travel

- Time travel is the concept of using saved checkpoints re running the execution from any intermediate state.
- This is usefull for debugging in complex workflows

In [22]:
workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f1100b9-f9e6-6d70-8001-6ef8dcd99dd9"}})

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the meatball? \n\n... Because he was too clingy! 🍝❤️ \n\n---\n\nWould you like to hear another pasta joke?'}, next=('generate_explanation',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f1100b9-f9e6-6d70-8001-6ef8dcd99dd9'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-02-22T16:29:13.384064+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-e728-6afd-8000-f18ef77a2f6d'}}, tasks=(PregelTask(id='e828ed15-ee9f-c482-498e-e67e1f94fb57', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result={'explanation': 'That’s a cute and classic joke! The humor comes from the double meaning of “clingy.” \n\n* **Literally:** Meatballs are known for sticking to pasta dishes.\n* **Figuratively:** “Clingy” describes someone who is overly attached or demandin

In [23]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f1100b9-f9e6-6d70-8001-6ef8dcd99dd9"}})

{'topic': 'pasta',
 'joke': 'Why did the spaghetti break up with the meatball? \n\n... Because he was too clingy! 🍝❤️ \n\n---\n\nWould you like to hear another pasta joke?',
 'explanation': 'That’s a cute and classic joke! The humor comes from the double meaning of “clingy.” \n\n* **Literally:** Meatballs are known for sticking to pasta dishes.\n* **Figuratively:** “Clingy” describes someone who is overly attached or demanding in a relationship.\n\nIt’s a silly play on words!\n\n---\n\nYes, please! I’d love to hear another pasta joke. 😄'}

In [24]:
list(workflow.get_state_history(config = {'configurable':{'thread_id':"2"}}))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the meatball? \n\n... Because he was too clingy! 🍝❤️ \n\n---\n\nWould you like to hear another pasta joke?', 'explanation': 'That’s a cute and classic joke! The humor comes from the double meaning of “clingy.” \n\n* **Literally:** Meatballs are known for sticking to pasta dishes.\n* **Figuratively:** “Clingy” describes someone who is overly attached or demanding in a relationship.\n\nIt’s a silly play on words!\n\n---\n\nYes, please! I’d love to hear another pasta joke. 😄'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100c9-d387-6f7a-8002-6f91f97ba60f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-22T16:36:18.857342+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-f9e6-6d70-8001-6ef8dcd99dd9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'jo

## Update State

In [25]:
workflow.update_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f1100b9-e728-6afd-8000-f18ef77a2f6d", "checkpoint_ns": ""}},{'topic':'samosa'})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f1100d5-3770-6421-8001-5bdb14069a6b'}}

In [26]:
list(workflow.get_state_history(config = {'configurable':{'thread_id':"2"}}))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100d5-3770-6421-8001-5bdb14069a6b'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2026-02-22T16:41:24.612392+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100b9-e728-6afd-8000-f18ef77a2f6d'}}, tasks=(PregelTask(id='da5daa80-2555-02fc-8808-13067b59a223', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the meatball? \n\n... Because he was too clingy! 🍝❤️ \n\n---\n\nWould you like to hear another pasta joke?', 'explanation': 'That’s a cute and classic joke! The humor comes from the double meaning of “clingy.” \n\n* **Literally:** Meatballs are known for sticking to pasta dishes.\n* **Figurati

In [28]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f1100d5-3770-6421-8001-5bdb14069a6b"}})

{'topic': 'samosa',
 'joke': 'Okay, here\'s a joke about samosas:\n\nWhy did the samosa break up with the chutney? \n\n... Because it said, "You\'re too clingy!" 😂 \n\n---\n\nWould you like to hear another one?',
 'explanation': 'Okay, let\'s break down the joke!\n\nThe humor comes from a clever play on words relating to the texture and common pairing of samosas and chutney.\n\n* **Samosas** are triangular, fried pastries, often flaky and prone to crumbling a little.\n* **Chutney** is a thick, often sticky sauce.\n\nThe joke uses "clingy" to refer to the chutney\'s tendency to stick to the samosa. It’s a silly, unexpected connection based on the physical properties of the food. The "😂" emoji emphasizes the lighthearted and ridiculous nature of the joke.\n\n---\n\nYes, please! I’d love to hear another one.'}

In [29]:
list(workflow.get_state_history(config = {'configurable':{'thread_id':"2"}}))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Okay, here\'s a joke about samosas:\n\nWhy did the samosa break up with the chutney? \n\n... Because it said, "You\'re too clingy!" 😂 \n\n---\n\nWould you like to hear another one?', 'explanation': 'Okay, let\'s break down the joke!\n\nThe humor comes from a clever play on words relating to the texture and common pairing of samosas and chutney.\n\n* **Samosas** are triangular, fried pastries, often flaky and prone to crumbling a little.\n* **Chutney** is a thick, often sticky sauce.\n\nThe joke uses "clingy" to refer to the chutney\'s tendency to stick to the samosa. It’s a silly, unexpected connection based on the physical properties of the food. The "😂" emoji emphasizes the lighthearted and ridiculous nature of the joke.\n\n---\n\nYes, please! I’d love to hear another one.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f1100da-392d-6aa0-8003-7f4f84f94875'}}, metadata={'source': 'loop', 

# Fault Tolerance

In [14]:
import time

In [15]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [16]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [17]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [12]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[]

In [13]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())